# 胶囊网络指南

本文参考来自 Kaggle 社区的英文[指南](https://www.kaggle.com/fizzbuzz/beginner-s-guide-to-capsule-networks/notebook)，翻译并补充内容。

本笔记本使用 Keras 介绍并实现胶囊网络 (CapsNet)，并在 DonorsChoose.Org 的[竞赛](https://www.kaggle.com/c/donorschoose-application-screening/overview)中评估了其性能。
文中提供的示例仅用于促进对模型的理解。

在 Kaggle Kernel 中运行时，因为获取数据集的需要可能需要**开启互联网连接**。

胶囊网络最初在 2017 年由 Hinton 等人的《Dynamic Routing Between Capsules》提出。

## 目录

- 胶囊网络介绍
    - 人类视觉识别
    - 动机：CNN 的缺陷
    - 胶囊
    - 按一致路由
    - 胶囊网络的数学推导
    - 动态路由算法
    - 关于压缩函数
    - 胶囊网络的优势
- 样例代码

## 胶囊网络介绍

### 人类视觉识别

任何现实的对象都可以认为是由几个较小的对象组成。例如，一棵树由树干，树冠和根组成。这些部分形成层次结构。一棵树的树冠进一步由树枝组成，树枝上有叶子。

![Parts of a tree](https://raw.githubusercontent.com/zaffnet/images/master/images/tree.png)

每当我们看到某个物体时，我们的眼睛就会注视一些固定点，这些固定点的相对位置和性质有助于我们的大脑识别该物体。这样，我们的大脑不必处理所有细节。仅仅看到一些树叶和树枝，我们的大脑就会意识到树冠。冠立在树干上，树干下面有一些根。结合这些分层信息，我们的大脑就会意识到有一棵树。由此，我们将对象的各个部分称为实体。

### 动机：CNN 的缺陷



### 胶囊

胶囊网络模型背后的假设是，存在胶囊（一组神经元）来告诉图像中是否存在某些对象（**实体**）。对应于每个实体，都有一个胶囊，它给出：

- 实体存在于图像中的概率
- 该实体的**实例化参数**(instantiation parameters)

实例化参数是图像中该实体的属性（例如“位置”、“大小”、“色相”等）。例如，矩形是一个简单的几何对象。对应于矩形的胶囊将告诉我们其实例化参数。

![Rectangle capsule](https://raw.githubusercontent.com/zaffnet/images/master/images/rectangle.png)

从上图中可见，我们假想的胶囊由 6 个神经元组成，每个神经元对应于矩形的某些属性，一组神经元的值对应一个向量，称为**激活向量**。该向量的长度表示的是矩阵存在的可能性。因此，矩形存在的可能性可写作：

$$
\sqrt{1.3^{2} + 0.6^{2} + 7.4^{2} + 6.5^{2} + 0.5^{2} + 1.4^{2}} = 10.06
$$

但是，如果使用概率来表示可能性，这个值应该在 0 到 1 的范围内。因此胶囊的输出需要进行如下转换：

$$
\begin{equation}
\mathbf{v}_{j}=\frac{\left\|\mathbf{s}_{j}\right\|^{2}}{1+\left\|\mathbf{s}_{j}\right\|^{2}}  \frac{\mathbf{s}_{j}}{\left\|\mathbf{s}_{j}\right\|}
\end{equation}
$$

这个非线性变换被称为压缩函数 (squashing function)，它充当胶囊网络的激活函数，如果 CNN 中使用的 ReLU。



### 按一致路由

一个胶囊网络由若干层组成。低层的胶囊对应于简单的实体（例如，矩形、三角形）。这些低级的胶囊推断存在更复杂的实体，并且它们的推断被“组合”以产生高级胶囊（例如，房屋的门、窗）的输出。例如，存在一个*矩形*（x轴角度 = 0，大小 = 5，位置 = 0，...）和*三角形*（x轴角度 = 6，大小 = 5，位置 = 5，...）共同推断一座*房屋*的存在（更高层的实体）。

需要注意的是，胶囊网络中，胶囊之间可能有**耦合作用** (coupling effect)。当一个低层级的胶囊成功学习到一个高层级实体时，该实体对应的高层级胶囊将反馈发送到这些低层级胶囊，从而增加其在该高级别胶囊上的推测。为理解这一点，我们假设有这样两层胶囊：

- 低层对应于矩形，三角形和圆形
- 高层对应于房屋，船只和汽车

如果一个图像包含房屋，则网络中对应于矩形和三角形的胶囊将具有较大的激活向量。它们的相对位置（编码在其实例化参数中）将取决于高级对象。由于它们将就房屋的存在达成一致，房屋胶囊的输出向量将变大。反过来，这将使矩形和三角形胶囊所得的预测值更大。此循环将重复 4-5 次，之后对房屋存在的推测概率将比对船或汽车存在的推测概率要大得多。

### 胶囊网络的数学推导

假设第 $l$ 层和第 $l + 1$ 层分别有 $m$ 和 $n$ 个胶囊，我们的任务是根据第 $l$ 层的激活值计算第 $l + 1$ 层的激活值。令 $u$ 表示第 $l$ 层的激活值，我们计算的是第 $l + 1$ 层的激活值 $v$。

对于第 $l + 1$ 层的胶囊 $j$：

(1) 首先通过第 $l$ 层计算**预测向量**。第 $l$ 层胶囊 $i$ 提供给第 $l + 1$ 层的胶囊 $j$ 的预测向量是：
$$
\hat{\mathbf{u}_{j | i}}=\mathbf{W}_{i j} \mathbf{u}_{i}
$$
$\mathbf{W}_{i j}$ 是待学习的权重矩阵。

(2) 然后计算胶囊 $j$ 的**输出向量**。输出向量是第 $l$ 层胶囊对胶囊 $j$ 给出的所有预测向量的加权和。
$$
s_{j}=\sum_{i=1}^{m} c_{i j} \hat{\mathbf{u}_{j | i}}
$$
标量 $\mathbf{c}_{ij}$ 被称为第 $l$ 层胶囊 $i$ 和第 $l + 1$ 层胶囊 $j$ 之间的耦合系数。这些系数由**迭代动态路由算法**确定。

(3) 对输出向量应用压缩函数，来获得胶囊 $j$ 的激活向量 $\mathbf{v}_j$：
$$
\mathbf{v}_{j}=\text{ squash}\left(\mathbf{s}_{j}\right)
$$

### 动态路由算法 The dynamic routing algorithm

路由算法 The dynamic routing algorithm

由 CNN 的缺陷得到启发，胶囊的概念并不复杂。构建胶囊网络的关键在于实现胶囊之间的耦合作用，动态路由算法解决了这一问题。

第 $l + 1$ 层的激活向量发送反馈信号到第 $l$ 层的胶囊。如果第 $l$ 层胶囊 $i$ 和第 $l + 1$ 层胶囊 $j$ 在胶囊 $j$ 的激活向量上的推测一致，它们的点积(dot product)值应该比较高。因此，预测向量 $\hat{\mathbf{u}_{j | i}}$ 的“宽度”在 $j$ 的输出中应该提升。换句话说，那些有助于激活向量的预测向量在输出向量（以及后续的激活向量）中具有更多的权重。这个互助周期持续 4-5 轮。

但是低层胶囊对高层胶囊的预测应该被加和到一个值，也就是为什么对于第 $l$ 层胶囊 $i$ 有：

$$
c_{i j}=\frac{\exp \left(b_{i j}\right)}{\sum_{k} \exp \left(b_{i k}\right)}
$$

显然，

$$
\sum_{k} c_{i k}=1
$$

其中，$c_{ij}$ 如前文所述，是两胶囊之间的耦合系数。logit $b_{ij}$ 表示第 $l$ 层胶囊 $i$ 和第 $l + 1$ 层胶囊 $j$ 是否有强耦合。换句话说，它是对胶囊 $j$ 的存在有多少程度是被胶囊 $i$ 解释的度量。所有 $b_ij$ 的初始值应该是相等的。

> 给定预测向量 $\hat{\mathbf{u}_{j | i}}$，路由迭代次数 $r$，
>
> 对所有第 $l$ 层胶囊 $i$ 和第 $l + 1$ 层胶囊 $j$，$b_{ij} = 0$。
>
> 对于 $r$ 次迭代：
>
>> 对第 $l$ 层的所有胶囊 $i$：$c_i=\text{softmax}(b_i)$ 一个胶囊对高层胶囊的推测值的和应为 1
>>
>> 对第 $l + 1$ 层的所有胶囊 $j$：$s_{j}=\sum_{i=1}^{m} c_{i j} \mathbf{u}_{j i}$，输出向量是预测向量的加权和
>>
>> 对第 $l + 1$ 层的所有胶囊 $j$：$\mathbf{v}_{j}=\operatorname{squash}\left(\mathbf{s}_{j}\right)$，应用激活函数
>>
>> 对所有第 $l$ 层胶囊 $i$ 和第 $l + 1$ 层胶囊 $j$：$b_{ij} = b_{ij} + \hat{\mathbf{u}_{j | i}} \cdot \mathbf{v}_j $
>
> 返回 $\mathbf{v}_j$

循环中的最后一行至关重要，路由就依此实现。如果积 $\hat{\mathbf{u}_{j | i}} \cdot \mathbf{v}_j$ 很大，它将增大 $b_{ij}$，这将进一步增大对应的耦合系数 $c_{ij}$，反过来会使 $\hat{\mathbf{u}_{j | i}} \cdot \mathbf{v}_j$ 变得更大。

这是胶囊网络工作的原理。了解这一点之后，你应该能没有困难地直接阅读[原论文](https://arxiv.org/pdf/1710.09829.pdf)。

### 关于压缩函数
当 $\|\mathbf{s}\| = 0$ 时，$\|\mathbf{s}\| $ 的导数是为定义的，它在训练期间可能会发生梯度爆炸。如果一个向量是零向量，它的梯度会是 `NaN`，所以当优化器更新这个变量时，它会变为 `NaN`。解决方案是通过计算平方和加上很小的 epsilon 值的平方根来手动计算范数：$\|\mathbf{s}\|\approx \sqrt{\sum_{i} s_{i}^{2}+\epsilon}$

### 胶囊网络的优势

CNN 当中包含池化层，我们通常使用最大池化来作为一种最原始的路由机制。一个局部池中，最活跃的特征会被路由到更高层，而高层的检测器在路由过程中没有任何反馈。相比于胶囊网络按一致路由(routing by agreement)的机制，只有与高层检测器达成一致的特征才会被路由。这是胶囊网络相比于 CNN 的关键优势，它具有出色的动态路由机制。称为动态，是因为需要路由的信息是实时确定的。

## 样例代码

In [ ]:
import gc
import os
import nltk
import tqdm
import numpy as np
import pandas as pd
nltk.download("punkt") # 载入该数据可能需要互联网连接！
# nltk 自然语言工具包，用于以 Python 编程语言编写的英语的符号和统计自然语言处理。包括图形演示和样本数据。

In [ ]:
def tokenize_sentences(sentences, words_dict):
    tokenized_sentences = []
    for sentence in tqdm.tqdm(sentences):
        if hasattr(sentence, "decode"):
            sentence = sentence.decode("utf-8")
        tokens = nltk.tokenize.word_tokenize(sentence)
        result = []
        for word in tokens:
            word = word.lower()
            if word not in words_dict:
                words_dict[word] = len(words_dict)
            word_index = words_dict[word]
            result.append(word_index)
        tokenized_sentences.append(result)
    return tokenized_sentences, words_dict

In [ ]:
def read_embedding_list(file_path):
    embedding_word_dict = {}
    embedding_list = []
    f = open(file_path)

    for index, line in enumerate(f):
        if index == 0:
            continue
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            continue
        embedding_list.append(coefs)
        embedding_word_dict[word] = len(embedding_word_dict)
    f.close()
    embedding_list = np.array(embedding_list)
    return embedding_list, embedding_word_dict

In [ ]:
def clear_embedding_list(embedding_list, embedding_word_dict, words_dict):
    cleared_embedding_list = []
    cleared_embedding_word_dict = {}

    for word in words_dict:
        if word not in embedding_word_dict:
            continue
        word_id = embedding_word_dict[word]
        row = embedding_list[word_id]
        cleared_embedding_list.append(row)
        cleared_embedding_word_dict[word] = len(cleared_embedding_word_dict)

    return cleared_embedding_list, cleared_embedding_word_dict

In [ ]:
def convert_tokens_to_ids(tokenized_sentences, words_list, embedding_word_dict, sentences_length):
    words_train = []

    for sentence in tokenized_sentences:
        current_words = []
        for word_index in sentence:
            word = words_list[word_index]
            word_id = embedding_word_dict.get(word, len(embedding_word_dict) - 2)
            current_words.append(word_id)

        if len(current_words) >= sentences_length:
            current_words = current_words[:sentences_length]
        else:
            current_words += [len(embedding_word_dict) - 1] * (sentences_length - len(current_words))
        words_train.append(current_words)
    return words_train

## 胶囊网络模型

除了增加了一个胶囊层，胶囊网络的架构与一般深度学习模型的架构非常相似。
在模型中使用胶囊层，而消除了池化层的存在。在许多情况下，这是比较理想的，因为模型保持了平移不变性，同时没有丢失细节。

![Text Classification](https://raw.githubusercontent.com/zaffnet/images/master/images/comparison.jpg)

In [ ]:
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.engine import Layer
from keras.layers import Activation, Add, Bidirectional, Conv1D, Dense, Dropout, Embedding, Flatten
from keras.layers import concatenate, GRU, Input, K, LSTM, MaxPooling1D
from keras.layers import GlobalAveragePooling1D,  GlobalMaxPooling1D, SpatialDropout1D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import text, sequence
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks

### 胶囊网络参数

In [ ]:
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.3
rate_drop_dense = 0.3

In [ ]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

### 胶囊网络层

In [ ]:
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [ ]:
def get_model(embedding_matrix, sequence_length, dropout_rate, recurrent_units, dense_size):
    input1 = Input(shape=(sequence_length,))
    embed_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=False)(input1)
    embed_layer = SpatialDropout1D(rate_drop_dense)(embed_layer)

    x = Bidirectional(
        GRU(gru_len, activation='relu', dropout=dropout_p, recurrent_dropout=dropout_p, return_sequences=True))(
        embed_layer)
    capsule = Capsule(num_capsule=Num_capsule, dim_capsule=Dim_capsule, routings=Routings,
                      share_weights=True)(x)
    capsule = Flatten()(capsule)
    capsule = Dropout(dropout_p)(capsule)
    output = Dense(1, activation='sigmoid')(capsule)
    model = Model(inputs=input1, outputs=output)
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])
    return model

In [ ]:
def _train_model(model, batch_size, train_x, train_y, val_x, val_y):
    num_labels = train_y.shape[1]
    patience = 5
    best_loss = -1
    best_weights = None
    best_epoch = 0
    
    current_epoch = 0
    
    while True:
        model.fit(train_x, train_y, batch_size=batch_size, epochs=1)
        y_pred = model.predict(val_x, batch_size=batch_size)

        total_loss = 0
        for j in range(num_labels):
            loss = log_loss(val_y[:, j], y_pred[:, j])
            total_loss += loss

        total_loss /= num_labels

        print("Epoch {0} loss {1} best_loss {2}".format(current_epoch, total_loss, best_loss))

        current_epoch += 1
        if total_loss < best_loss or best_loss == -1:
            best_loss = total_loss
            best_weights = model.get_weights()
            best_epoch = current_epoch
        else:
            if current_epoch - best_epoch == patience:
                break

    model.set_weights(best_weights)
    return model

In [ ]:
def train_folds(X, y, X_test, fold_count, batch_size, get_model_func):
    print("="*75)
    fold_size = len(X) // fold_count
    models = []
    result_path = "predictions"
    if not os.path.exists(result_path):
        os.mkdir(result_path)
    for fold_id in range(0, fold_count):
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size

        if fold_id == fold_size - 1:
            fold_end = len(X)

        train_x = np.concatenate([X[:fold_start], X[fold_end:]])
        train_y = np.concatenate([y[:fold_start], y[fold_end:]])

        val_x = np.array(X[fold_start:fold_end])
        val_y = np.array(y[fold_start:fold_end])

        model = _train_model(get_model_func(), batch_size, train_x, train_y, val_x, val_y)
        train_predicts_path = os.path.join(result_path, "train_predicts{0}.npy".format(fold_id))
        test_predicts_path = os.path.join(result_path, "test_predicts{0}.npy".format(fold_id))
        train_predicts = model.predict(X, batch_size=512, verbose=1)
        test_predicts = model.predict(X_test, batch_size=512, verbose=1)
        np.save(train_predicts_path, train_predicts)
        np.save(test_predicts_path, test_predicts)

    return models

## 训练

In [ ]:
# train_file_path = "../input/donorschooseorg-preprocessed-data/train_preprocessed.csv"
train_file_path = "../input/donorschooseorg-preprocessed-data/train_small.csv"

# test_file_path = "../input/donorschooseorg-preprocessed-data/test_preprocessed.csv"
test_file_path = "../input/donorschooseorg-preprocessed-data/test_small.csv"

# embedding_path = "../input/fatsttext-common-crawl/crawl-300d-2M/crawl-300d-2M.vec"
embedding_path = "../input/donorschooseorg-preprocessed-data/embeddings_small.vec"

batch_size = 128 # 256
recurrent_units = 16 # 64
dropout_rate = 0.3 
dense_size = 8 # 32
sentences_length = 10 # 300
fold_count = 2 # 10

In [ ]:
UNKNOWN_WORD = "_UNK_"
END_WORD = "_END_"
NAN_WORD = "_NAN_"
CLASSES = ["project_is_approved"]

In [ ]:
# 
print("Loading data...")
train_data = pd.read_csv(train_file_path)   # 训练数据
test_data = pd.read_csv(test_file_path)     # 测试数据
list_sentences_train = train_data["application_text"].fillna(NAN_WORD).values
list_sentences_test = test_data["application_text"].fillna(NAN_WORD).values
y_train = train_data[CLASSES].values

In [ ]:
print("Tokenizing sentences in train set...")
tokenized_sentences_train, words_dict = tokenize_sentences(list_sentences_train, {})
print("Tokenizing sentences in test set...")
tokenized_sentences_test, words_dict = tokenize_sentences(list_sentences_test, words_dict)

In [ ]:
# Embedding
words_dict[UNKNOWN_WORD] = len(words_dict)
print("Loading embeddings...")
embedding_list, embedding_word_dict = read_embedding_list(embedding_path)
embedding_size = len(embedding_list[0])

In [ ]:
print("Preparing data...")
embedding_list, embedding_word_dict = clear_embedding_list(embedding_list, embedding_word_dict, words_dict)

embedding_word_dict[UNKNOWN_WORD] = len(embedding_word_dict)
embedding_list.append([0.] * embedding_size)
embedding_word_dict[END_WORD] = len(embedding_word_dict)
embedding_list.append([-1.] * embedding_size)

embedding_matrix = np.array(embedding_list)

id_to_word = dict((id, word) for word, id in words_dict.items())
train_list_of_token_ids = convert_tokens_to_ids(
    tokenized_sentences_train,
    id_to_word,
    embedding_word_dict,
    sentences_length)
test_list_of_token_ids = convert_tokens_to_ids(
    tokenized_sentences_test,
    id_to_word,
    embedding_word_dict,
    sentences_length)
X_train = np.array(train_list_of_token_ids)
X_test = np.array(test_list_of_token_ids)

In [ ]:
get_model_func = lambda: get_model(
    embedding_matrix,
    sentences_length,
    dropout_rate,
    recurrent_units,
    dense_size)

In [ ]:
del train_data, test_data, list_sentences_train, list_sentences_test
del tokenized_sentences_train, tokenized_sentences_test, words_dict
del embedding_list, embedding_word_dict
del train_list_of_token_ids, test_list_of_token_ids
gc.collect();

In [ ]:
print("Starting to train models...")
models = train_folds(X_train, y_train, X_test, fold_count, batch_size, get_model_func)

## 提交

使用默认参数以 10 折训练模型。使用平均排名进行提交。

In [ ]:
from scipy.stats import rankdata

LABELS = ["project_is_approved"]

base = "../input/donorschooseorg-application-screening-predictions/predictions/predictions/"
predict_list = []
for j in range(10):
    predict_list.append(np.load(base + "predictions_001/test_predicts%d.npy"%j))
    
print("Rank averaging on ", len(predict_list), " files")
predcitions = np.zeros_like(predict_list[0])
for predict in predict_list:
    predcitions = np.add(predcitions.flatten(), rankdata(predict)/predcitions.shape[0])  
predcitions /= len(predict_list)

submission = pd.read_csv('../input/donorschoose-application-screening/sample_submission.csv')
submission[LABELS] = predcitions
submission.to_csv('submission.csv', index=False)

## 结论

我们可以看到，胶囊网络有助于文本分类。即使没有任何超参数调整，也可以使用胶囊网络建立强大的基准。

## 参考

- https://www.kaggle.com/fizzbuzz/beginner-s-guide-to-capsule-networks/notebook